In [1]:
!pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client langchain-google-genai



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import base64
from typing import TypedDict, Annotated, List, Literal
from email.mime.text import MIMEText

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Google API Imports
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [25]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import datetime


In [6]:
# 1. SETUP GMAIL AUTHENTICATION
SCOPES = [
    'https://www.googleapis.com/auth/gmail.modify',
    'https://www.googleapis.com/auth/calendar'
]


def get_gmail_service():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials1.json', SCOPES
            )
            creds = flow.run_local_server(port=0)

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    gmail_service = build('gmail', 'v1', credentials=creds)
    calendar_service = build('calendar', 'v3', credentials=creds)

    return gmail_service, calendar_service


service, calendar_service = get_gmail_service()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=384278549136-eiug4ucl4ivh7tukmbqqtupv4b68bbhi.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52342%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=QgNU4wIFyqR1WTGNZ4CaNctCk5GVzU&access_type=offline


In [26]:
def create_event(calendar_service, summary, start_dt, end_dt, attendees=None):
    if not summary or not start_dt or not end_dt:
        print("Skipping event creation: missing required info")
        return None

    event = {
        "summary": summary,
        "start": {"dateTime": start_dt, "timeZone": "Asia/Kolkata"},
        "end": {"dateTime": end_dt, "timeZone": "Asia/Kolkata"},
    }

    if attendees:
        event["attendees"] = [{"email": email} for email in attendees]

    try:
        created_event = calendar_service.events().insert(
            calendarId='primary', body=event).execute()
        print("Event created:", created_event.get('htmlLink'))
        return created_event
    except HttpError as error:
        print("Calendar API error:", error)
        return None


In [27]:
import os
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file(
            'token.json', SCOPES
        )

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credential.json',
                SCOPES,
                redirect_uri='urn:ietf:wg:oauth:2.0:oob'  
            )

            auth_url, _ = flow.authorization_url(
                prompt='consent'
            )

            print("\n🔗 OPEN THIS URL IN YOUR BROWSER:\n")
            print(auth_url)

            code = input("\n📌 Paste the authorization code here: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

service = get_gmail_service()


In [28]:
def read_inbox(service, max_results=5):
    results = service.users().messages().list(
        userId='me',
        labelIds=['INBOX'],
        maxResults=max_results
    ).execute()

    messages = results.get('messages', [])

    if not messages:
        print("No messages found.")
        return

    for msg in messages:
        msg_id = msg['id']
        message = service.users().messages().get(
            userId='me',
            id=msg_id,
            format='metadata',
            metadataHeaders=['From', 'Subject']
        ).execute()

        headers = message['payload']['headers']
        email_data = {}

        for h in headers:
            email_data[h['name']] = h['value']

        print("\n📩 New Email")
        print("From:", email_data.get('From'))
        print("Subject:", email_data.get('Subject'))


In [29]:
read_inbox(service)


📩 New Email
From: fatimi bee <fatimibee312@gmail.com>
Subject: Project Update

📩 New Email
From: fatimi bee <fatimibee312@gmail.com>
Subject: Interview Discussion

📩 New Email
From: fatimi bee <fatimibee312@gmail.com>
Subject: Application for Internship Opportunity

📩 New Email
From: fatimi bee <fatimibee312@gmail.com>
Subject: Request for Information

📩 New Email
From: Fatimi Bee <b230955@skit.ac.in>
Subject: regarding request


In [30]:

import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables from .env
load_dotenv()

# Read API key
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

# Create OpenAI client
client = OpenAI(api_key=api_key)

try:
    # List models available for this key
    models = client.models.list()

    print("Models available for this key:")
    for model in models.data:
        print(model.id)

except Exception as e:
    print("Error:", e)


Models available for this key:
gpt-3.5-turbo
gpt-5.2-codex
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
chatgpt-image-latest
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-0125
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
gpt-4o-audio-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-mini-tts
o3-2025-04-16
o4-mini-2025-04-16
o3
o4-mini
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano

In [31]:
! pip install langchain langgraph langchain-openai python-dotenv 



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install -U langchain-openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:
def load_llm():
    from dotenv import load_dotenv
    import os
    from langchain_openai import ChatOpenAI
  # LangChain OpenAI chat model

    # Load environment variables from .env
    load_dotenv()
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # Make sure this is set in your .env

    if not OPENAI_API_KEY:
        raise ValueError("Please set OPENAI_API_KEY in your .env file")

    # Initialize OpenAI LLM via LangChain
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",  # free-tier model
        temperature=0,
        openai_api_key=OPENAI_API_KEY
    )

    return llm


In [33]:
def build_email_prompt(user_prompt: str):
    from langchain_core.prompts import (
        ChatPromptTemplate,
        SystemMessagePromptTemplate,
        HumanMessagePromptTemplate
    )

    system_text = f"""
You are an AI email assistant.

User instructions:
{user_prompt}

IMPORTANT RULES:
- Output STRICT JSON only
- Do NOT use markdown
- Do not add names or signatures

JSON FORMAT:
The AI must return a JSON object with these keys:
- action: string, one of "reply", "ignore", or "schedule"
- reply: string, the plain text email reply (only if action is "reply")
- event: object with keys "title", "date", "start_time", "end_time" (only if action is "schedule")
"""

    system_msg = SystemMessagePromptTemplate.from_template(system_text)

    human_msg = HumanMessagePromptTemplate.from_template(
        "Subject: {subject}\nFrom: {sender}\nBody: {body}"
    )

    return ChatPromptTemplate.from_messages([system_msg, human_msg])


In [34]:
import base64
from email.mime.text import MIMEText

def fetch_unread_emails(service, max_results=5):
    return service.users().messages().list(
        userId="me",
        labelIds=["UNREAD"],
        maxResults=max_results
    ).execute().get("messages", [])

def extract_email(service, msg_id):
    msg = service.users().messages().get(
        userId="me",
        id=msg_id,
        format="full"
    ).execute()

    headers = msg["payload"]["headers"]
    subject = sender = ""

    for h in headers:
        if h["name"] == "Subject":
            subject = h["value"]
        elif h["name"] == "From":
            sender = h["value"]

    body = ""
    payload = msg["payload"]

    if "parts" in payload:
        for part in payload["parts"]:
            if part["mimeType"] == "text/plain":
                data = part["body"].get("data")
                if data:
                    body = base64.urlsafe_b64decode(data).decode()
    else:
        data = payload["body"].get("data")
        if data:
            body = base64.urlsafe_b64decode(data).decode()

    return subject, sender, body, msg["threadId"]

import re

def clean_email_body(body: str) -> str:
    # Remove markdown JSON blocks
    body = re.sub(r"```json[\s\S]*?```", "", body, flags=re.IGNORECASE)

    # Remove quoted replies (Gmail style)
    body = re.split(r"\nOn .* wrote:", body)[0]

    # Remove lines starting with >
    body = "\n".join(
        line for line in body.splitlines()
        if not line.strip().startswith(">")
    )

    return body.strip()

def send_reply(service, to, subject, reply, thread_id):
    message = MIMEText(reply)
    message["to"] = to
    message["subject"] = "Re: " + subject

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()

    service.users().messages().send(
        userId="me",
        body={"raw": raw, "threadId": thread_id}
    ).execute()

def mark_read(service, msg_id):
    service.users().messages().modify(
        userId="me",
        id=msg_id,
        body={"removeLabelIds": ["UNREAD"]}
    ).execute()


In [36]:
import json

def process_all_unread_emails(service, user_prompt, calendar_service=None, max_results=20):
    """
    Process unread emails using AI and take actions (reply, schedule, ignore).

    Args:
        service: Gmail API service object
        user_prompt: Base instruction for AI
        calendar_service: Optional, Google Calendar service for scheduling
        max_results: Number of emails to fetch
    """
    # Build prompt and LLM
    prompt = build_email_prompt(user_prompt)
    llm = load_llm()

    # Fetch unread emails
    emails = fetch_unread_emails(service, max_results=max_results)
    print(f"\n📬 Total unread emails: {len(emails)}\n")

    if not emails:
        print("🎉 No unread emails.")
        return

    for idx, msg in enumerate(emails, start=1):
        print("=" * 70)
        print(f"📧 Processing email {idx}/{len(emails)}")

        # Extract email details
        subject, sender, body, thread_id = extract_email(service, msg["id"])
        body = clean_email_body(body)

        print(f"From   : {sender}")
        print(f"Subject: {subject}")
        print("Body preview:")
        print(body[:300])
        print("=" * 70)

        # ---- AI RESPONSE ----
        try:
            ai_response = llm.invoke(
                prompt.format_prompt(
                    subject=subject,
                    sender=sender,
                    body=body,
                    action="Reply"  # Required placeholder
                ).to_messages()
            ).content
        except KeyError as e:
            print(f"❌ KeyError in prompt formatting: {e}. Skipping email.")
            continue
        except Exception as e:
            print(f"❌ AI invocation failed: {e}. Skipping email.")
            continue

        # Parse AI JSON safely
        try:
            result = json.loads(ai_response)
        except json.JSONDecodeError:
            print("❌ AI returned invalid JSON. Skipping email.")
            continue

        action = result.get("action", "ignore")  # Default to ignore
        reply = result.get("reply", "")

        # ---- Handle scheduled event ----
        if action == "schedule":
            if calendar_service is None:
                print("❌ Calendar service not provided. Cannot schedule event.")
                continue

            event = result.get("event", {})
            start_dt = f"{event.get('date','')}T{event.get('start_time','00:00')}:00"
            end_dt = f"{event.get('date','')}T{event.get('end_time','00:00')}:00"

            create_event(
                calendar_service,
                title=event.get("title", "Meeting"),
                description=body,
                start_dt=start_dt,
                end_dt=end_dt
            )

            mark_read(service, msg["id"])
            print("✅ Event scheduled & email marked as read")
            continue

        # ---- Handle ignored email ----
        if action == "ignore":
            print("🚫 AI chose to ignore this email.")
            continue

        # ---- Draft reply ----
        print("\n🤖 AI Draft Reply")
        print("-" * 50)
        print(reply)
        print("-" * 50)

        decision = input("Send this reply? (y / edit / skip): ").lower()

        if decision == "y":
            send_reply(service, sender, subject, reply, thread_id)
            mark_read(service, msg["id"])
            print("✅ Reply sent & marked as read")

        elif decision == "edit":
            edited = input("\n✏️ Enter your edited reply:\n")
            send_reply(service, sender, subject, edited, thread_id)
            mark_read(service, msg["id"])
            print("✅ Edited reply sent & marked as read")

        else:
            print("⏭️ Skipped (email remains unread)")


In [37]:
USER_PROMPT = """
Reply only to genuine emails.
Ignore promotions and job ads.
Be polite, concise, and human-like.
Do not add name or signature.
"""

process_all_unread_emails(service, USER_PROMPT)



📬 Total unread emails: 1

📧 Processing email 1/1
From   : fatimi bee <fatimibee312@gmail.com>
Subject: Interview Discussion
Body preview:
Hi,

We would like to schedule an interview call with you on
5th February at 10:00 AM.

Please confirm your availability.

Thanks,
HR Team

🤖 AI Draft Reply
--------------------------------------------------
Hi,

Thank you for the invitation. I confirm my availability for an interview call on February 5th at 10:00 AM.

Looking forward to it.
--------------------------------------------------
✅ Reply sent & marked as read


In [38]:
def get_state_schema():
    from typing import TypedDict, Optional

    class EmailState(TypedDict):
        subject: str
        sender: str
        body: str
        thread_id: str
        decision: dict
        final_reply: Optional[str]

    return EmailState


In [39]:
def ai_decision_node(state, prompt, llm):
    import json
    import re

    chain = prompt | llm

    response = chain.invoke({
        "subject": state["subject"],
        "sender": state["sender"],
        "body": state["body"]
    })

    raw = response.content.strip()

    # 🔑 REMOVE ```json ``` wrappers
    raw = re.sub(r"^```json\s*", "", raw)
    raw = re.sub(r"^```\s*", "", raw)
    raw = re.sub(r"\s*```$", "", raw)

    try:
        decision = json.loads(raw)
    except Exception as e:
        print("❌ JSON parse failed:", e)
        print("RAW OUTPUT:\n", response.content)
        decision = {"action": "ignore"}

    return {"decision": decision}


In [40]:
def human_confirmation_node(state):
    reply = state["decision"].get("reply")

    print("\n🤖 AI Draft Reply")
    print("-" * 50)
    print(reply)
    print("-" * 50)

    while True:
        choice = input("Send reply? (y / n / edit): ").strip().lower()

        if choice == "y":
            return {"final_reply": reply}

        if choice == "n":
            return {"final_reply": None}

        if choice == "edit":
            print("\n✏️ Enter edited reply (empty line to finish):")
            lines = []
            while True:
                line = input()
                if not line:
                    break
                lines.append(line)
            return {"final_reply": "\n".join(lines)}


In [41]:
def calendar_node_factory(calendar_service):
    def calendar_node(state):
        event = state["decision"].get("event", {})

        start_dt = f"{event['date']}T{event['start_time']}:00"
        end_dt = f"{event['date']}T{event['end_time']}:00"

        create_event(
            calendar_service,
            title=event.get("title", "Meeting"),
            description=state["body"],
            start_dt=start_dt,
            end_dt=end_dt
        )

        return {}

    return calendar_node


In [42]:
def send_node_factory(service):
    def send_node(state):
        if state.get("final_reply"):
            send_reply(
                service,
                state["sender"],
                state["subject"],
                state["final_reply"],
                state["thread_id"]
            )
            print("✅ Reply sent")
        else:
            print("⏭️ Skipped")

        return {}
    return send_node


In [43]:
def build_email_graph(user_prompt: str, service, calendar_service):
    from langgraph.graph import StateGraph, END

    EmailState = get_state_schema()
    llm = load_llm()
    prompt = build_email_prompt(user_prompt)

    graph = StateGraph(EmailState)

    graph.add_node(
        "ai_decision",
        lambda s: ai_decision_node(s, prompt, llm)
    )

    graph.add_node("human_confirm", human_confirmation_node)
    graph.add_node("send", send_node_factory(service))
    graph.add_node("calendar", calendar_node_factory(calendar_service))

    graph.set_entry_point("ai_decision")

    graph.add_conditional_edges(
        "ai_decision",
        lambda s: s["decision"]["action"],
        {
            "reply": "human_confirm",
            "ignore": END,
            "schedule": "calendar"
        }
    )

    graph.add_edge("human_confirm", "send")
    graph.add_edge("send", END)
    graph.add_edge("calendar", END)

    return graph.compile()


In [48]:
def run_email_assistant(service, user_prompt, max_results=3):
    # Build LangGraph app
    app = build_email_graph(user_prompt, service, calendar_service)


    # Fetch unread emails
    emails = fetch_unread_emails(service, max_results=max_results)

    if not emails:
        print("📭 No unread emails found.")
        return

    print(f"\n📬 Total unread emails: {len(emails)}")

    for i, msg in enumerate(emails, start=1):
        print("\n" + "=" * 70)
        print(f"📧 Processing email {i}/{len(emails)}")

        subject, sender, body, thread_id = extract_email(service, msg["id"])
        body = clean_email_body(body)

        print("From   :", sender)
        print("Subject:", subject)
        print("Body preview:")
        print(body[:500])
        print("=" * 70)

        # Initial state for graph
        state = {
            "subject": subject,
            "sender": sender,
            "body": body,
            "thread_id": thread_id,
        }

        # Run LangGraph
        app.invoke(state)

        # Mark as read ONLY after graph completes
        mark_read(service, msg["id"])
        print("📩 Email marked as read")


In [55]:
USER_PROMPT = """
Reply only to genuine emails.
Ignore promotions and job ads.
Schedule meetings if the email discusses a meeting, call, interview, or appointment.
"""


run_email_assistant(service, USER_PROMPT)



📬 Total unread emails: 1

📧 Processing email 1/1
From   : fatimi bee <fatimibee312@gmail.com>
Subject: Project Discussion Meeting
Body preview:
Hi Saniya,

I hope you are doing well. Can we schedule a meeting tomorrow at 3:00 PM
IST to discuss the Infosys Email Assistant project progress?
Please let me know if that time works for you or suggest another slot.

Thanks,
Fatimi


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}